In [1]:
import pandas as pd
import requests
import csv
from dotenv import load_dotenv
import os

In [2]:
load_dotenv() 
API_KEY = os.getenv("API_PORTAL_TRANSPARENCIA")

In [3]:
def buscar_contrato(siafi_orgao, num_contrato):
    base_url = "https://api.portaldatransparencia.gov.br/api-de-dados/contratos"
    
    # Cabeçalhos da requisição
    headers = {
    "chave-api-dados": API_KEY,
    "Accept": "application/json",  # Define o formato da resposta
    }

    # Inicializando variáveis
    pagina = 1
    contratos_encontrados = []

    # Loop para buscar todas as páginas
    while True:
        # Monta a URL com os parâmetros
        url = f"{base_url}?codigoOrgao={siafi_orgao}&pagina={pagina}"
        
        # Faz a requisição
        response = requests.get(url, headers=headers)
    
        # Verifica se a resposta foi bem-sucedida
        if response.status_code != 200:
            print(f"Erro na requisição: {response.status_code}")
            print(response.text)
            break  # Interrompe o loop em caso de erro
    
        # Converte a resposta para JSON
        data = response.json()
    
        # Se não houver mais dados, interrompe o loop
        if not data:
            break
    
        # Filtra os contratos pelo número desejado
        for contrato in data:
            if contrato.get("numero") == num_contrato:
                contratos_encontrados.append(contrato)
    
        # Avança para a próxima página
        pagina += 1
    
    # Exibe os contratos encontrados
    if contratos_encontrados:
        print(f"Foram encontrados {len(contratos_encontrados)} contratos com número {num_contrato}:")
        for contrato in contratos_encontrados:
            print(contrato)
    else:
        print(f"Nenhum contrato encontrado com número {num_contrato}.")

In [4]:
def buscar_contrato_fornecedor(siafi_orgao, nome_empresa):
    base_url = "https://api.portaldatransparencia.gov.br/api-de-dados/contratos"  
    
    # Cabeçalhos da requisição
    headers = {
    "chave-api-dados": API_KEY,  # 07/02
    "Accept": "application/json",  # Define o formato da resposta
    }
    
    # Inicializando variáveis
    pagina = 1
    contratos_encontrados = []

    # Loop para buscar todas as páginas
    while True:
        # Monta a URL com os parâmetros
        url = f"{base_url}?codigoOrgao={siafi_orgao}&pagina={pagina}"
    
        # Faz a requisição
        response = requests.get(url, headers=headers)
    
        # Verifica se a resposta foi bem-sucedida
        if response.status_code != 200:
            print(f"Erro na requisição: {response.status_code}")
            print(response.text)
            break  # Interrompe o loop em caso de erro
    
        # Converte a resposta para JSON
        data = response.json()
    
        # Se não houver mais dados, interrompe o loop
        if not data:
            break
    
        # Filtra os contratos pelo código do órgão e nome da empresa
        for contrato in data:
            # Pegando o código SIAFI correto
            orgao_codigo = contrato.get("unidadeGestora", {}).get("orgaoVinculado", {}).get("codigoSIAFI", "")
    
            # Pegando o nome da empresa corretamente
            fornecedor = contrato.get("fornecedor", {})
            nome_fornecedor = fornecedor.get("nome", "").strip()
            razao_social = fornecedor.get("razaoSocialReceita", "").strip()
    
            # Verifica se o contrato pertence ao órgão correto
            if orgao_codigo == siafi_orgao:
                # Verifica se o fornecedor é o desejado
                if nome_fornecedor == nome_empresa or razao_social == nome_empresa:
                    contratos_encontrados.append({
                        "Número do Contrato": contrato.get("numero", ""),
                        "Objeto": contrato.get("objeto", ""),
                        "Data de Assinatura": contrato.get("dataAssinatura", ""),
                        "Valor Inicial": contrato.get("valorInicialCompra", ""),
                        "Valor Final": contrato.get("valorFinalCompra", ""),
                        "Código SIAFI": orgao_codigo,
                        "Fornecedor": nome_fornecedor or razao_social
                    })
    
        # Avança para a próxima página
        pagina += 1
    
    # Exibe os contratos encontrados
    if contratos_encontrados:
        print(f"Foram encontrados {len(contratos_encontrados)} contratos da empresa '{nome_empresa}' no órgão '{siafi_orgao}':")
    
        df_resultados = pd.DataFrame(contratos_encontrados)
        print(df_resultados)
    else:
        print(f"Nenhum contrato encontrado para a empresa '{nome_empresa}' no órgão '{siafi_orgao}'.")

In [5]:
def buscar_contrato_por_cnpj(siafi_orgao, cnpj_formatado):
    base_url = "https://api.portaldatransparencia.gov.br/api-de-dados/contratos"  

    # Cabeçalhos da requisição
    headers = {
        "chave-api-dados": API_KEY,  # Substitua se necessário
        "Accept": "application/json",
    }

    pagina = 1
    contratos_encontrados = []

    while True:
        url = f"{base_url}?codigoOrgao={siafi_orgao}&pagina={pagina}"
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"❌ Erro na requisição: {response.status_code}")
            print(response.text)
            break

        try:
            data = response.json()
        except ValueError:
            print("❌ Erro ao converter resposta para JSON.")
            break

        if not data:
            break

        for contrato in data:
            orgao_codigo = contrato.get("unidadeGestora", {}).get("orgaoVinculado", {}).get("codigoSIAFI", "")
            fornecedor = contrato.get("fornecedor", {})
            cnpj_api = fornecedor.get("cnpjFormatado", "").strip()

            if orgao_codigo == siafi_orgao and cnpj_api == cnpj_formatado:
                contratos_encontrados.append({
                    "Número do Contrato": contrato.get("numero", ""),
                    "Objeto": contrato.get("objeto", ""),
                    "Data de Assinatura": contrato.get("dataAssinatura", ""),
                    "Valor Inicial": contrato.get("valorInicialCompra", ""),
                    "Valor Final": contrato.get("valorFinalCompra", ""),
                    "Código SIAFI": orgao_codigo,
                    "CNPJ Fornecedor": cnpj_api,
                    "Fornecedor": fornecedor.get("nome", "") or fornecedor.get("razaoSocialReceita", "")
                })

        pagina += 1

    if contratos_encontrados:
        print(f"✅ {len(contratos_encontrados)} contratos encontrados para o CNPJ '{cnpj_formatado}' no órgão '{siafi_orgao}'.")
        return pd.DataFrame(contratos_encontrados)
    else:
        print(f"⚠️ Nenhum contrato encontrado para o CNPJ '{cnpj_formatado}' no órgão '{siafi_orgao}'.")
        return pd.DataFrame([])

In [5]:
buscar_contrato('000032021', '25000')

Nenhum contrato encontrado com número 25000.


In [24]:
buscar_contrato_fornecedor("36212", "FBX - SERVICOS DE SEGURANCA LTDA")

Foram encontrados 1 contratos da empresa 'FBX - SERVICOS DE SEGURANCA LTDA' no órgão '36212':
  Número do Contrato                                             Objeto  \
0          000012018  Objeto: CONTRATACAO DE EMPRESA PARA PRSTAR SER...   

  Data de Assinatura  Valor Inicial  Valor Final Código SIAFI  \
0         2018-06-01       512400.0   -480010.08        36212   

                         Fornecedor  
0  FBX - SERVICOS DE SEGURANCA LTDA  


In [6]:
buscar_contrato_por_cnpj('25000', '39.420.336/0001-49')

⚠️ Nenhum contrato encontrado para o CNPJ '39.420.336/0001-49' no órgão '25000'.


""


In [6]:
df_total = pd.read_csv('../data/terceirizados/arquivos_tratados/df_total.csv', low_memory=False)

In [7]:
df_total.head(20)

,id_terc,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_contrato,nr_cnpj,nm_razao_social,nr_cpf,nm_terceirizado,...,nm_unidade_prestacao,vl_mensal_salario,vl_mensal_custo,Num_Mes_Carga,Mes_Carga,Ano_Carga,sg_orgao,nm_orgao,cd_orgao_siafi,cd_orgao_siape
0,7971733,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,12018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.906.681-**,LUIS CARLOS PONTES DA SILVA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
1,7971734,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,12018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.394.463-**,FRANCISCO JOSE SILVA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
2,7971735,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,12018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.243.351-**,MICHEL CHRISTIAN CORREA DE BARROS,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
3,7971736,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,12018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.094.541-**,WENDEL BERGUE DE SOUSA MIRANDA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
4,7971737,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,12018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.355.123-**,RAFAEL TEIXEIRA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
5,7971738,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,12018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.429.743-**,SERGIO CASTRO SILVA,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
6,7971739,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,12018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.351.113-**,IZAU LIMA DOS SANTOS,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
7,7971740,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,12018,10869440000133,ATIVA BRIGADISTA LTDA ME,***.861.401-**,ALESSANDRO CAMPOS DOS SANTOS,...,CAPITAL FINANCIAL CENTER,2829.080078125,7545.64013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
8,7971741,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,22017,09284699000133,AVAL EMPRESA DE SEGURANCA LTDA- EPP.,***.747.661-**,EDIGAR LEANDRO PEREIRA SANTOS,...,BIBLIOTECA DEMONSTRATIVA DE BRASILIA,2616.300048828125,5642.89013671875,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000
9,7971742,M.CIDADA,550005,COORDENACAO GERAL DE LOGISTICA E ADMINISTRACAO,C.G.L.C./ M.D.S.,22017,09284699000133,AVAL EMPRESA DE SEGURANCA LTDA- EPP.,***.646.221-**,FABIO RODRIGUES MOREIRA,...,BIBLIOTECA DEMONSTRATIVA DE BRASILIA,2983.110107421875,6319.240234375,5,MAIO,2019,MDS,MINISTERIO DO DESENVOLVIMENTO SOCIAL,55000,55000


In [8]:
df_total[df_total['nm_razao_social'] == 'RM SERVILOS EMPRESARIAIS LTDA-ME'].iloc[0].to_string()

'id_terc                                                                8047178\nsg_orgao_sup_tabela_ug                                                MINISTER\ncd_ug_gestora                                                           253029\nnm_ug_tabela_ug              COORDENACAO ESTADUAL DE VIGILANCIA SANITARIA D...\nsg_ug_gestora                                                 CVSPAF-AC/ANVISA\nnr_contrato                                                              12012\nnr_cnpj                                                         09646758000176\nnm_razao_social                               RM SERVILOS EMPRESARIAIS LTDA-ME\nnr_cpf                                                          ***.235.682-**\nnm_terceirizado                                      HELIO DANGELO DE OLIVEIRA\nnm_categoria_profissional               782305 - MOTORISTA DE CARRO DE PASSEIO\nnm_escolaridade                                          ENSINO MEDIO COMPLETO\nnr_jornada                             

In [23]:
buscar_contrato("36212", "000012013")

Foram encontrados 1 contratos com número 000012013:
{'id': 668526938, 'numero': '000012013', 'objeto': 'Objeto: CESSÃO DE USO NÃO ONEROSA DE UMA ÁREA DE 131,03M², A TITULO GRATUITO, LOCALIZADO NO PAVIMENTO SUPERIOR DO PRÉDIO DO NÚCLEO DE APOIO PORTUÁRIO - NAP, NO PORTO DE FORTALEZA, CONSTRUÍDO EM ÁREA DA CEDENTE E O RATEIO DE DESPESAS DECORRENTES DA OCUPAÇÃO DO ESPAÇO FÍSICO CEDIDO A CESSIONÁRIA, CONFORME O DISPOSTO NA LEGISLAÇÃO EM VIGOR.', 'numeroProcesso': 'Sem informação', 'fundamentoLegal': '', 'compra': {'numero': '000022021', 'objeto': 'Objeto: Termo de Cessão de Uso não Onerosa de áreas e rateio de despesas entre a CVPAF/CE e a Companhia Docas-Porto de Fortaleza.', 'numeroProcesso': '25763443616201355', 'contatoResponsavel': 'RANIELE FERREIRA DE LIMA'}, 'situacaoContrato': 'Não se aplica', 'modalidadeCompra': 'Inexigibilidade de Licitação', 'unidadeGestora': {'codigo': '253002', 'nome': 'AGENCIA NACIONAL DE VIGILANCIA SANITARIA', 'descricaoPoder': 'EXECUTIVO', 'orgaoVinculado':

In [7]:
# Busca pelo contrato 37/2021 da Anatel-PE
buscar_contrato("41231", "000372021")

Foram encontrados 1 contratos com número 000372021:
{'id': 668680939, 'numero': '000372021', 'objeto': 'Objeto: PRESTAÇÃO DE SERVIÇO DE VIGILÂNCIA PATRIMONIAL PARA A ANATEL PERNAMBUCO', 'numeroProcesso': 'Sem informação', 'fundamentoLegal': '', 'compra': {'numero': '000022021', 'objeto': 'Objeto: Pregão Eletrônico -  Contratação de serviço de vigilância patrimonial para a Anatel Pernambuco, conforme condições, quantidades e exigências estabelecidas no Edital e seus anexos, para o prazo inicial de 20 (vinte) meses', 'numeroProcesso': '53532001468202021', 'contatoResponsavel': 'FELIPE DA MOTA PAZZOLA'}, 'situacaoContrato': 'Não se aplica', 'modalidadeCompra': 'Pregão', 'unidadeGestora': {'codigo': '413007', 'nome': 'AGENCIA NACIONAL DE TELECOMUNICACOES-PE', 'descricaoPoder': 'EXECUTIVO', 'orgaoVinculado': {'codigoSIAFI': '41231', 'cnpj': '02030715000112', 'sigla': 'ANATEL', 'nome': 'Agêncial Nacional de Telecomunicações'}, 'orgaoMaximo': {'codigo': '41000', 'sigla': 'COMUNICAÇÕES', 'nome

In [8]:
# Busca pelo contrato jun/2018 da DPRF
buscar_contrato("30802", "000062018")

Foram encontrados 12 contratos com número 000062018:
{'id': 668494285, 'numero': '000062018', 'objeto': 'Objeto: CONTRATAÇÃO DE SERVIÇOS DE JARDINAGEM,LIMPEZA E CONSERVAÇÃO, BEM COMO DE SERVIÇOS CORRELATOS, COM FORNECIMENTO DE INSUMOS, ACESSÓRIOS E TODOS OS EQUIPAMENTOS NECESSÁRIOS, JUNTO AACADEMIA NACIONAL DA POLÍCIA RODOVIÁRIA FEDERAL.', 'numeroProcesso': 'Sem informação', 'fundamentoLegal': 'Fundamento Legal: LEI 8.666/93, LEI 10.520/02, DEC. 2.271/97 E LEGISLAÇÃO C', 'compra': {'numero': '000012018', 'objeto': 'Objeto: Pregão Eletrônico -  Contratação da prestação de serviços continuados de jardinagem, limpeza e conservação para as dependências da Academia Nacional da Polícia Rodoviária Federal em Florianópolis. Contrato de 44 horas semanais. Demais especificações no Termo de Referência e Edital do certame.', 'numeroProcesso': '08812001637201816', 'contatoResponsavel': 'MARCELO QUINTANILHA ADILINO'}, 'situacaoContrato': 'Não se aplica', 'modalidadeCompra': 'Pregão', 'unidadeGestora

In [10]:
# Buscar contrato 110000/2016 do INCRA-GOIANIA/GO
buscar_contrato_fornecedor("22201", "APECE SERVICOS GERAIS LTDA")

Foram encontrados 2 contratos da empresa 'APECE SERVICOS GERAIS LTDA' no órgão '22201':
  Número do Contrato                                             Objeto  \
0          130002011  Objeto: Prestação do serviços de limpeza, cons...   
1          110002016  Objeto: O OBJETO DO PRESENTE INSTRUMENTO É A C...   

  Data de Assinatura  Valor Inicial  Valor Final Código SIAFI  \
0         2011-12-27      307573.44    781621.20        22201   
1         2016-12-05      378450.00    451422.84        22201   

                   Fornecedor  
0  APECE SERVICOS GERAIS LTDA  
1  APECE SERVICOS GERAIS LTDA  


In [8]:
buscar_contrato_fornecedor("25201", "ENGEAR ENGENHARIA DE AQUECIMENTO E REFRIGERACAO LTDA")

Erro na requisição: 401
{"Erro na API":"Chave de API inválida!"}
Nenhum contrato encontrado para a empresa 'ENGEAR ENGENHARIA DE AQUECIMENTO E REFRIGERACAO LTDA' no órgão '25201'.
